In [2]:
import tensorflow as tf
import numpy as np
from transformers import CLIPProcessor, CLIPModel

# Load pre-trained CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")


In [4]:
import os 
os.listdir("./")
files = os.listdir()

In [10]:
from PIL import Image

In [55]:
import tensorflow as tf

# Example image and text pairs
image_paths = [files[4], files[1]]
text_descriptions = ["person wearing red shirt", "person with backpack"]

# Encode images
image_features = []
for image_path in image_paths:
    image_norm = tf.io.read_file(image_path)
    image_norm = tf.image.decode_jpeg(image_norm, channels=3)
    image_norm = tf.image.resize(image_norm, (224, 224))  # Resize to CLIP input size
    image_norm = tf.cast(image_norm, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    image_norm = clip_processor(images=image_norm, return_tensors="tf")
    image_features.append(image_norm)

# Encode text descriptions
text_features = clip_processor(text_descriptions, return_tensors="tf", padding=True, truncation=True)
# print(text_features)
# Ensure text features have compatible shape
# text_features["input_ids"] = tf.tile(text_features["input_ids"], [len(image_paths), 1])  # Repeat text features for each image
# print(text_features["input_ids"])
# print(image_features)
# Compute similarity scores
similarity_scores = []
for i in range(len(image_paths)):
    image_feature = image_features[i]["pixel_values"]
    text_feature = tf.cast(text_features["input_ids"][i], dtype=image_feature.dtype)  # Cast text feature to the same dtype as image feature
    # print(text_feature)
    # similarity = tf.matmul(image_feature, tf.reshape(text_feature, (1,1,1,6)))
    reshaped_text_feature = tf.reshape(text_feature, (1, 1, 6, 1))  # Add dimensions
    reshaped_text_feature = tf.broadcast_to(reshaped_text_feature, (1, 6, 224, 224))  # Broadcast
    similarity = tf.matmul(image_feature, reshaped_text_feature, transpose_b=True)
    # similarity = tf.matmul(image_feature, tf.expand_dims(text_feature, axis=-1))  # Expand dims to make text feature compatible for matmul
    similarity_scores.append(similarity.numpy()[0][0])

# Print similarity scores
for i, score in enumerate(similarity_scores):
    print(f"Image {i+1} - Text similarity score: {score:.4f}")

# You can now use these similarity scores for person re-identification.
# Higher scores indicate better matches between images and text descriptions.


InvalidArgumentError: {{function_node __wrapped__BroadcastTo_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [1,1,6,1] vs. [1,6,224,224] [Op:BroadcastTo]

In [28]:
# Compute similarity scores
similarity_scores = []
for i in range(len(image_paths)):
    image_feature = image_features[i]["pixel_values"]
    
    # Extract a single text feature corresponding to the current image
    text_feature = tf.expand_dims(text_features["input_ids"][i], axis=0)
    
    # Debugging: Print the shapes of image_feature and text_feature
    print("Image Feature Shape:", image_feature.shape)
    print("Text Feature Shape:", text_feature.shape)
    
    # Ensure text_feature has the expected shape
    assert text_feature.shape[0] == 1, "Text feature should have shape [1, num_features]"
    
    similarity = tf.matmul(image_feature, tf.expand_dims(text_feature, axis=-1))
    similarity_scores.append(similarity.numpy()[0][0])

# Print similarity scores
for i, score in enumerate(similarity_scores):
    print(f"Image {i+1} - Text similarity score: {score:.4f}")
resize

Image Feature Shape: (1, 3, 224, 224)
Text Feature Shape: (1, 6)


InvalidArgumentError: cannot compute _MklBatchMatMulV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:BatchMatMulV2] name: 

In [58]:
!pip install imgbeddings

In [60]:
files

['.ipynb_checkpoints',
 'backpack.jpeg',
 'coop_model1.ipynb',
 'GNR.ipynb',
 'red_shirt_guy.jpg']

In [61]:
import requests
from PIL import Image
from imgbeddings import imgbeddings

img1 = Image.open(files[4])
img2 = Image.open(files[1])
ibed = imgbeddings()
embeddings = [ibed.to_embeddings(img1), ibed.to_embeddings(img2)]

C:\Users\nawka\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:649: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


C:\Users\nawka\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nawka\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\nawka\anaconda3\Lib\site-packages\transformers\models\clip\processing_clip.py:143: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


In [67]:
# tf.experimental.numpy.experimental_enable_numpy_behavior()

text_inputs = clip_processor(text_descriptions, return_tensors="tf", padding=True, truncation=True)
text_features = clip_model.get_text_features(**text_inputs)

# Compute similarity scores between image embeddings and text features
similarity_scores = tf.matmul(image_embeddings, text_features, transpose_b=True)

# Get the most similar text description for each image
top_matches = tf.argmax(similarity_scores, axis=1)

# Print the results
for i, match_idx in enumerate(top_matches):
    print(f"Image {i+1}: {text_descriptions[match_idx]}")

TypeError: 'numpy.int32' object is not callable